<a href="https://colab.research.google.com/github/kevinhhl/options-pricing-tools-and-trading-strategies/blob/main/Implied_Volatility_Lookup_tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yahoo_fin
!pip install yfinance

In [2]:
import time
import pandas as pd
from datetime import date
from yahoo_fin import options
import yfinance as yf
import numpy as np

# **Overview:**

A common way to predict future realized volatility is by directly obtaining implied volatility (IV) from the market. 

This is arguably more reliable than building models to forecast it based on historical volatility. 


How to obtain IV from the market:


* **Method 1:**

 Using the Black Scholes Model, set the price of the option equal to the quoted market price, then holding all variables constant and letting IV as the unknown variable, solve for sigma. The solved value is our IV.
 
 When solving for sigma, we can validate whether implied volatility is correct by: (1) checking that the model's price is being closely equal to the option's market price, and (2) the delta should be extremely close to 50. 


* **Method 2** (*Shortcut*): 

 There is actually no need for us to solve IV by ourselves through coding. The option chain provided by Yahoo Finance has the IVs for each individual contract. We just need to look up options that are at-the-money or near-the-money.

This script will use Method 2. We will go further by looking up the IVs of an option at the same strike price but across different maturities. This information will be useful when considering or calendar spread strategies, or perhaps to build further models to understand the term structure of options for specific markets and specific sectors.

In [3]:
# Manual inputs:
symbol                   = "TSLA" 
riskfree_rate            = 0.0513
date_today               = date(2023,2,20)  # for crnt_price

In [4]:
ticker_yahoo = yf.Ticker(symbol)
data = ticker_yahoo.history()
crnt_price = data['Close'].iloc[-1]
crnt_price

208.30999755859375

In [5]:
def _get_iv(chain):
  ''' get_iv(Object)
      @param Object : chain, an object from options.get_options_chain()
      Returns implied volatility as float. 
  '''
  iv = None
  for i in range(len(chain)-1):
      c = chain["Strike"]
      if c[i+1] >= crnt_price and c[i] <= crnt_price:
        iv = chain["Implied Volatility"][i]
        break
  return float(iv.strip("%"))/100


In [6]:
def get_iv(type_options):
  ''' get_iv(string)
      @param string : type_options, either "calls" or "puts"
      Returns tuple of (A) list of expiry dates, and (B) list of IV
  '''
  list_iv = [] 
  list_expdates= []
  for date_expire_str in options.get_expiration_dates(symbol):
    tmp = options.get_options_chain(symbol, date_expire_str)[type_options]
    iv = _get_iv(tmp)
    list_expdates.append(date_expire_str)
    list_iv.append(iv)
    print("{}; ATM Call; iv={}".format(date_expire_str, round(iv,4)))
  return (list_expdates, list_iv)


In [7]:
list1, list2 = get_iv("calls")
pd.DataFrame({ "Options":list1, "IV":list2 }, columns=["Options", "IV"] )

February 24, 2023; ATM Call; iv=0.8252
March 3, 2023; ATM Call; iv=0.884
March 10, 2023; ATM Call; iv=0.8014
March 17, 2023; ATM Call; iv=0.7637
March 24, 2023; ATM Call; iv=0.7382
March 31, 2023; ATM Call; iv=0.7206
April 21, 2023; ATM Call; iv=0.7344
May 19, 2023; ATM Call; iv=0.7098
June 16, 2023; ATM Call; iv=0.6873
July 21, 2023; ATM Call; iv=0.6838
September 15, 2023; ATM Call; iv=0.667
December 15, 2023; ATM Call; iv=0.6642
January 19, 2024; ATM Call; iv=0.6598
March 15, 2024; ATM Call; iv=0.6657
June 21, 2024; ATM Call; iv=0.6533
September 20, 2024; ATM Call; iv=0.6588
January 17, 2025; ATM Call; iv=0.6549
June 20, 2025; ATM Call; iv=0.6469


,Options,IV
0,"February 24, 2023",0.8252
1,"March 3, 2023",0.8840
2,"March 10, 2023",0.8014
3,"March 17, 2023",0.7637
4,"March 24, 2023",0.7382
5,"March 31, 2023",0.7206
6,"April 21, 2023",0.7344
7,"May 19, 2023",0.7098
8,"June 16, 2023",0.6873
9,"July 21, 2023",0.6838


In [8]:
np.average(np.array(list2))

0.7121666666666666

In [9]:
list1, list2 = get_iv("puts")
pd.DataFrame({ "Options":list1, "IV":list2 }, columns=["Options", "IV"] )

February 24, 2023; ATM Call; iv=0.8091
March 3, 2023; ATM Call; iv=0.8568
March 10, 2023; ATM Call; iv=0.7747
March 17, 2023; ATM Call; iv=0.7308
March 24, 2023; ATM Call; iv=0.7048
March 31, 2023; ATM Call; iv=0.6858
April 21, 2023; ATM Call; iv=0.6871
May 19, 2023; ATM Call; iv=0.6512
June 16, 2023; ATM Call; iv=0.6201
July 21, 2023; ATM Call; iv=0.6065
September 15, 2023; ATM Call; iv=0.5767
December 15, 2023; ATM Call; iv=0.554
January 19, 2024; ATM Call; iv=0.5443
March 15, 2024; ATM Call; iv=0.5394
June 21, 2024; ATM Call; iv=0.5173
September 20, 2024; ATM Call; iv=0.5109
January 17, 2025; ATM Call; iv=0.5044
June 20, 2025; ATM Call; iv=0.4853


,Options,IV
0,"February 24, 2023",0.8091
1,"March 3, 2023",0.8568
2,"March 10, 2023",0.7747
3,"March 17, 2023",0.7308
4,"March 24, 2023",0.7048
5,"March 31, 2023",0.6858
6,"April 21, 2023",0.6871
7,"May 19, 2023",0.6512
8,"June 16, 2023",0.6201
9,"July 21, 2023",0.6065


In [10]:
np.average(np.array(list2))

0.6310666666666668